In [1]:
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 
from pathlib import Path


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'valid/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 50

paths = Path(args.train_folder).glob('**/chorale_*.csv')
train_np_list = [pd.read_csv(p).values.tolist() for p in paths]
# print(len(train_np_list[0]))

paths = Path(args.val_folder).glob('**/chorale_*.csv')
val_np_list = [pd.read_csv(p).values.tolist() for p in paths]

paths = Path(args.test_folder).glob('**/chorale_*.csv')
test_np_list = [pd.read_csv(p).values.tolist() for p in paths]

print(train_np_list[0])

[[66, 61, 57, 54], [66, 61, 57, 54], [68, 61, 59, 54], [68, 61, 59, 54], [69, 66, 61, 54], [69, 66, 61, 56], [69, 66, 61, 57], [69, 66, 61, 59], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 59, 49], [68, 65, 59, 49], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 59, 50], [66, 66, 59, 50], [68, 66, 59, 50], [68, 66, 59, 50], [69, 66, 61, 49], [69, 66, 61, 49], [69, 66, 61, 47], [69, 66, 61, 47], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 59, 49], [68, 65, 59, 49], [66, 61, 57, 42], [66, 61, 57, 42], [66, 61, 57, 42], [66, 61, 57, 42], [73, 66, 57, 54], [73, 66, 57, 54], [73, 66, 57, 54], [73, 66, 57, 54], [71, 66, 59, 56], [71, 66, 59, 56], [71, 65, 59, 56], [71, 65, 59, 56], [69, 66, 61, 57], [69, 66, 61, 57], [69, 66, 61, 59], [69, 66, 61, 59], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 61, 49], [69, 66, 61, 54], [69, 66, 61, 54], [69, 66, 61, 52], [69, 66, 

 ### Baseline

In [26]:
def baseline():
    def build_dataset_baseline(chorales):
        min_len = len(min(chorales, key=len))
        print(min_len)

        def reshape(chorale):
            chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
            return chorale
        def trunc(chorale):
            return chorale[:min_len+1]

        def target(chorale):
            X = chorale[:-1]
            Y = chorale[-1]
            return X, Y

        ragged_chorales = tf.ragged.constant(chorales)
        dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
        dataset = dataset.map(reshape)
        dataset = dataset.map(trunc)
        dataset = dataset.map(target)
        dataset = dataset.batch(32)
    #     for d in dataset:
    #         tf.print(d)
    #         break
        return dataset


    train_dataset = build_dataset_baseline(train_np_list)
    val_dataset = build_dataset_baseline(val_np_list)
    test_dataset = build_dataset_baseline(test_np_list)
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(1, input_shape=[None, 1])
    ])
    model.compile(loss='mse',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=[keras.metrics.mean_squared_error])
    model.fit(train_dataset, epochs=args.epochs, validation_data=val_dataset)
    
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
        keras.layers.SimpleRNN(20, return_sequences=True),
        keras.layers.SimpleRNN(1),
    ])
    model.compile(loss='mse',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=[keras.metrics.mean_squared_error])
    model.fit(train_dataset, epochs=args.epochs, validation_data=val_dataset)
    for d in val_dataset:
        tf.print(np.argmax(model.predict(d[0]), axis=1))
    
baseline()

100
128
128
Epoch 1/7
8/8 [==============================] - 1s 31ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 2/7
8/8 [==============================] - 0s 16ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 3/7
8/8 [==============================] - 0s 15ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 4/7
8/8 [==============================] - 0s 14ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 5/7
8/8 [==============================] - 0s 15ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - val_mean_squared_error: 5196.8164
Epoch 6/7
8/8 [==============================] - 0s 14ms/step - loss: 5163.3491 - mean_squared_error: 5163.3491 - val_loss: 5196.8164 - va

2021-10-06 16:25:33,079 : WARNING : 5 out of the last 7 calls to <function Model.make_predict_function.<locals>.predict_function at 0x15816d040> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


### Sparse categorical entropy

In [18]:
import sys

def min_max_notes(all_dataset_chorales=()):
    min_note = None
    max_note = None
    def _min_max_notes():
        nonlocal min_note
        nonlocal max_note
        if min_note:
            return min_note, max_note
        min_note = sys.maxsize
        max_note = -sys.maxsize - 1
        for chorales in all_dataset_chorales:
            for ch in chorales:
                ch = np.array(ch)
                min_note = ch[ch > 0].min() if ch[ch > 0].min() < min_note else min_note
                max_note = ch.max() if ch.max() > max_note else max_note
            
        return min_note, max_note

    return _min_max_notes()

min_note, max_note = min_max_notes((train_np_list, val_np_list, test_np_list))
num_notes = (max_note - min_note + 1) + 1 # different notes + 0 (no note) note
(min_note, max_note)

(36, 81)

In [25]:
def sparse_categorical_entropy():
    def build_dataset_sce(chorales):
        min_len = len(min(chorales, key=len))
        def reshape(chorale):
            chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
            return chorale
        def trunc(chorale):
            return chorale[:min_len+1]
        def transform(chorale):
            return tf.where(chorale == 0, 0, chorale - min_note)
        def target(chorale):
            X = chorale[:-1]
            Y = chorale[-1]
            return X, Y

        ragged_chorales = tf.ragged.constant(chorales)
        dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
        dataset = dataset.map(reshape)
        dataset = dataset.map(trunc)
        dataset = dataset.map(transform)
        dataset = dataset.map(target)
        dataset = dataset.batch(32)
        return dataset


    train_dataset = build_dataset_sce(train_np_list)
    val_dataset = build_dataset_sce(val_np_list)
    test_dataset = build_dataset_sce(test_np_list)
    
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
        keras.layers.SimpleRNN(20),
        keras.layers.Dense(num_notes, activation="softmax")
    ])
    for d in train_dataset:
        tf.print(model(d[0]).shape)
        break
    
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=['accuracy'])
    model.fit(train_dataset, epochs=50, validation_data=val_dataset)
    
    for d in val_dataset:
        tf.print(np.argmax(model.predict(d[0]), axis=1))
        
    
sparse_categorical_entropy()

TensorShape([32, 47])
Epoch 1/50
8/8 [==============================] - 2s 55ms/step - loss: 3.3852 - accuracy: 0.0830 - val_loss: 3.0887 - val_accuracy: 0.1184
Epoch 2/50
8/8 [==============================] - 0s 27ms/step - loss: 2.7737 - accuracy: 0.1397 - val_loss: 2.7287 - val_accuracy: 0.1184
Epoch 3/50
8/8 [==============================] - 0s 27ms/step - loss: 2.5678 - accuracy: 0.1747 - val_loss: 2.6641 - val_accuracy: 0.1053
Epoch 4/50
8/8 [==============================] - 0s 27ms/step - loss: 2.4856 - accuracy: 0.1790 - val_loss: 2.6112 - val_accuracy: 0.1053
Epoch 5/50
8/8 [==============================] - 0s 27ms/step - loss: 2.4425 - accuracy: 0.1834 - val_loss: 2.5866 - val_accuracy: 0.1053
Epoch 6/50
8/8 [==============================] - 0s 28ms/step - loss: 2.4185 - accuracy: 0.1921 - val_loss: 2.6465 - val_accuracy: 0.0789
Epoch 7/50
8/8 [==============================] - 0s 28ms/step - loss: 2.4045 - accuracy: 0.1790 - val_loss: 2.5806 - val_accuracy: 0.1053
Epoch

### Sparse categorical entropy. Larger datasets

In [ ]:
def sparse_categorical_entropy_larger_dataset():
    def build_dataset_sceld(chorales):
        min_len = len(min(chorales, key=len))
        def reshape(chorale):
            chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
            return chorale
        def trunc(chorale):
            return chorale[:min_len+1]
        def transform(chorale):
            return tf.where(chorale == 0, 0, chorale - min_note)
        def target(chorale):
            X = chorale[:-1]
            Y = chorale[-1]
            return X, Y

        ragged_chorales = tf.ragged.constant(chorales)
        dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
        dataset = dataset.map(reshape)
        dataset = dataset.map(trunc)
        dataset = dataset.map(transform)
        dataset = dataset.map(target)
        dataset = dataset.batch(32)
        return dataset


    train_dataset = build_dataset_sceld(train_np_list)
    val_dataset = build_dataset_sceld(val_np_list)
    test_dataset = build_dataset_sceld(test_np_list)
    
#     model = keras.models.Sequential([
#         keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
#         keras.layers.SimpleRNN(20),
#         keras.layers.Dense(num_notes, activation="softmax")
#     ])
#     for d in train_dataset:
#         tf.print(model(d[0]).shape)
#         break
    
#     model.compile(loss='sparse_categorical_crossentropy',
#                  optimizer=keras.optimizers.Nadam(learning_rate=0.01),
#                  metrics=['accuracy'])
#     model.fit(train_dataset, epochs=50, validation_data=val_dataset)
    
#     for d in val_dataset:
#         tf.print(np.argmax(model.predict(d[0]), axis=1))
        
    
sparse_categorical_entropy_larger_dataset()